In [25]:
import pandas as pd
import numpy as np
import osmnx as ox
pd.set_option('display.max_columns', 500)

In [26]:
north = -6.9875
west = -37.3303
south = -7.0882
east = -37.2215
pois = ox.geometries.geometries_from_bbox(north, south, east, west, tags={'amenity':True})
pois = pois[['amenity', 'name', 'geometry', 'opening_hours']]
pois.head()

amenity                        name  \
element_type osmid                                            
node         1798496605    fuel                 Posto Viola   
             1798496642    taxi  Moto táxi Praça dos Pombos   
             1798525769    taxi       Táxi Rodoviária Velha   
             1798525772    taxi    Táxi Fórum Miguel Sátiro   
             1798919196     bar                 Bar do Rock   

                                           geometry  \
element_type osmid                                    
node         1798496605  POINT (-37.27847 -7.02390)   
             1798496642  POINT (-37.27843 -7.02361)   
             1798525769  POINT (-37.27735 -7.02265)   
             1798525772  POINT (-37.27702 -7.02601)   
             1798919196  POINT (-37.27702 -7.02204)   

                                                      opening_hours  
element_type osmid                                                   
node         1798496605                                         NaN  
             1798496642                                         NaN  
             1798525769                                         NaN  
             1798525772                                         NaN  
             1798919196  We-Su 19:00-02:00+; week 25-26 18:00-04:00

In [27]:
# pois.to_json()
pois.to_file('pois_from_patos.geojson', driver="GeoJSON") 

In [28]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200", http_auth=('elastic', 'changeme'), verify_certs=False)

/tmp/ipykernel_15359/3962933453.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts="http://localhost:9200", http_auth=('elastic', 'changeme'), verify_certs=False)


In [29]:
import geojson
with open('pois_from_patos.geojson') as f:
    pois_list = geojson.load(f)
pois_list = pois_list['features']
dict(pois_list[0])

{'type': 'Feature',
 'geometry': {"coordinates": [-37.278472, -7.023898], "type": "Point"},
 'properties': {'element_type': 'node',
  'osmid': 1798496605,
  'amenity': 'fuel',
  'name': 'Posto Viola',
  'opening_hours': None}}

In [19]:
es.indices.create(index='patos-pois-index', ignore=400)

/tmp/ipykernel_15359/178293735.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='patos-pois-index', ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'patos-pois-index'})

In [32]:
es.info()

ObjectApiResponse({'name': 'elk', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'q7oWD5V7RWyCXRr4UqUt9g', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [30]:
for i,poi in enumerate(pois_list):
    resp = es.index(index="patos-pois-index", id=i+1, document=poi)
    if i%1000==0:
        print(str(i+1)+'º instance: ',resp['result'])
print('Completed.')

1º instance:  updated
Completed.


In [31]:
pois.shape

(829, 4)

In [34]:
es.search(body={"query": {"match_all": {}}}, index = 'patos-pois-index')

/tmp/ipykernel_15359/2386377839.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.search(body={"query": {"match_all": {}}}, index = 'patos-pois-index')


ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 829, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'patos-pois-index', '_id': '1', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.278472, -7.023898]}, 'properties': {'element_type': 'node', 'osmid': 1798496605, 'amenity': 'fuel', 'name': 'Posto Viola', 'opening_hours': None}}}, {'_index': 'patos-pois-index', '_id': '2', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.27843, -7.023605]}, 'properties': {'element_type': 'node', 'osmid': 1798496642, 'amenity': 'taxi', 'name': 'Moto táxi Praça dos Pombos', 'opening_hours': None}}}, {'_index': 'patos-pois-index', '_id': '3', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.277349, -7.022649]}, 'properties': {'element_type': 'node', 'osmid': 1798

In [36]:
es.indices.get_mapping(index = 'patos-pois-index')

ObjectApiResponse({'patos-pois-index': {'mappings': {'properties': {'geometry': {'properties': {'coordinates': {'type': 'float'}, 'type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}, 'properties': {'properties': {'amenity': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'element_type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'name': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'opening_hours': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'osmid': {'type': 'long'}}}, 'type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}})

In [ ]:
mappings = {
        "properties": {
            "type": {"type": "text", "analyzer": "english"},
            "title": {"type": "text", "analyzer": "english"},
            "director": {"type": "text", "analyzer": "standard"},
            "cast": {"type": "text", "analyzer": "standard"},
            "country": {"type": "text", "analyzer": "standard"},
            "date_added": {"type": "text", "analyzer": "standard"},
            "release_year": {"type": "integer"},
            "rating": {"type": "text", "analyzer": "standard"},
            "duration": {"type": "text", "analyzer": "standard"},
            "listed_in": {"type": "keyword"},
            "description": {"type": "text", "analyzer": "english"}
    }
}

es.indices.create(index="test-index", mappings=mappings)

In [ ]:
query = {
    "bool": {
        "filter": {
            "match_phrase": {
                "country": "United States"
            }
        },
        "must_not": {
            "range": {
                "release_year": {"lte": 2015}
            }
        }
    }
}

result = es.search(index="netflix_shows", query=query)
print(result)

In [ ]:
es.delete(index="netflix_shows", id="1337")

In [ ]:
es.indices.delete(index="netflix_shows")